In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
import pandas as pd
import tensorflow
import matplotlib.pyplot as plt
import numpy as np
from keras.layers.core import Dense, Dropout
from keras.layers import GRU
from tensorflow.keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax,Nadam
from keras.callbacks import Callback, CSVLogger
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from keras.models import Sequential
import timeit
from numpy import mean
from numpy import std
from numpy import dstack
from sklearn.preprocessing import StandardScaler
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
start = timeit.default_timer()
dataframe = pd.read_csv('/content/drive/MyDrive/Research/data/LaminatedPlate_RZT_Q9.csv')

In [ ]:
dataset = dataframe.values
data = dataset[:, 0:107]
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# data=scaler.fit_transform(data)

In [ ]:
nb_set = 50000
nb_test = int(nb_set*0.8)
x_train = data[:nb_test,0:101]
x_test  = data[nb_test:,0:101]
print(np.shape(x_train))
print('x_train = ', x_train)

y_train = data[:nb_test,101:107]
y_test  = data[nb_test:,101:107]
print(np.shape(y_train))
print('y_train = ',y_train)

(40000, 101)
x_train =  [[0.35280948 0.40013556 0.59792331 ... 0.25230566 0.16814912 0.60556815]
 [0.09663441 0.26536996 0.79194769 ... 0.89902329 0.86419623 0.14169278]
 [0.0186391  0.72832428 0.81665907 ... 0.03038468 0.63536209 0.33557237]
 ...
 [0.31114235 0.76524773 0.69068545 ... 0.29214632 0.22587294 0.76841789]
 [0.99450549 0.08655289 0.95196613 ... 0.67749678 0.26933526 0.38782586]
 [0.36902632 0.80323233 0.96519961 ... 0.02714565 0.27050196 0.6524935 ]]
(40000, 5)
y_train =  [[0.42717531 0.58751401 0.54305166 0.44421383 0.45133866]
 [0.40574368 0.51058079 0.67147243 0.46764448 0.37195867]
 [0.35747964 0.64572483 0.67587772 0.34620174 0.35082198]
 ...
 [0.2701243  0.73861164 0.76651982 0.23882662 0.25368718]
 [0.4142306  0.62034412 0.66766787 0.37549006 0.35519023]
 [0.4390913  0.52040345 0.59644907 0.46718325 0.4322217 ]]


In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

input_node  = x_train.shape[1]
output_node = y_train.shape[1]
n_timesteps = x_train.shape[2]

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64,kernel_size=5, activation='relu', input_shape=(input_node,n_timesteps)))
model.add(Conv1D(filters=64,kernel_size=5, activation='relu'))
model.add(Dropout(0.2))
model.add(GRU(units=200,return_sequences=True,activation="relu"))
model.add(GRU(units=100,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(output_node))
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model1 = model
model1.summary()


In [ ]:
json_string = model.to_json()
open('/content/drive/MyDrive/Noise_Complete_FGMBmodel.json', 'w').write(json_string)

4796

In [ ]:
#optimizers
adam = Adam(learning_rate=0.08* tpu_strategy.num_replicas_in_sync, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
rms = RMSprop(learning_rate = 0.01* tpu_strategy.num_replicas_in_sync, rho=0.9, epsilon=1e-08)
ada = Adagrad(learning_rate=0.2* tpu_strategy.num_replicas_in_sync, epsilon=1e-08)
sgd = SGD(learning_rate=0.1* tpu_strategy.num_replicas_in_sync, momentum=0.1)
adadel=Adadelta(learning_rate=1.0* tpu_strategy.num_replicas_in_sync, rho=0.95, epsilon=1e-08)
adamax = Adamax(learning_rate=0.002* tpu_strategy.num_replicas_in_sync, beta_1=0.9, beta_2=0.999)
nadam = Nadam(learning_rate=0.002* tpu_strategy.num_replicas_in_sync, beta_1=0.9, beta_2=0.999)
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
# fit network
csv_logger = CSVLogger('training_history.csv')
history = model.fit(x_train, y_train, epochs = 1000, validation_data=(x_test, y_test),batch_size=256* tpu_strategy.num_replicas_in_sync, verbose=1,callbacks=[csv_logger])
scores = model.evaluate(x_test, y_test,verbose=0)
model.save_weights('drive/My Drive/target_weight.h5', overwrite=True)
print(model.evaluate(x_train, y_train))

In [ ]:
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)
mse_train = mean_squared_error(pred_train, y_train)
mse_test  = mean_squared_error(pred_test, y_test)
mae_train = mean_absolute_error(pred_train, y_train)
mae_test = mean_absolute_error(pred_test,  y_test)

stop = timeit.default_timer()
with open("MSE_Computingtime.txt", "w") as text_file:
    print("Computing time: "+str(stop - start), file=text_file)
    print('MSE for train: ' +str(mse_train),file=text_file)
    print('MSE for test: '  + str(mse_test), file=text_file)
    print('mean absolute error for train: '+str(mae_train),file=text_file)
    print('mean absolute error for test: ' + str(mae_test), file=text_file)
    print('scores: ' +str(scores),file=text_file)
    print("%s:%.2f%%" % (model.metrics_names[1], scores[1] * 100),file=text_file)


print('Computing time:',stop - start)
print('mean square error for train and test',mse_train, mse_test)
print('scores ',scores)
print("%s:%.2f%%"%(model.metrics_names[1],scores[1]*100))

# #summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='lower right')
# plt.show()

# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper right')
# plt.show()

model.summary()
config = model.get_config()
model.get_weights()
print('config',config)
print(ascii(model))